In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ClaseAbdon import DataFrameAnalyzer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import statsmodels.api as sm
from sklearn.feature_selection import SelectFromModel

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer,normalize,RobustScaler,MaxAbsScaler

from sklearn.model_selection import cross_val_score
from sklearn.metrics import root_mean_squared_error

from sklearn.decomposition import PCA
import re

import snappy

import warnings
warnings.filterwarnings('ignore')

In [19]:
df2 = pd.read_parquet('data/train_juan.parquet',engine='pyarrow')

In [25]:
df2

,id,class,cap_shape,cap_surface,cap_color,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat,bruises
0,367,edible,bell,scaly,yellow,almond,free,close,broad,black,...,white,white,partial,white,one,pendant,brown,scattered,grasses,Bruises
1,5123,poisonous,convex,smooth,red,foul,free,close,narrow,buff,...,white,pink,partial,white,one,evanescent,white,several,paths,No presenta
2,2864,edible,convex,fibrous,green,null,free,close,broad,pink,...,pink,white,partial,white,one,pendant,brown,several,wood,Bruises
3,4674,poisonous,flat,smooth,white,foul,free,close,broad,pink,...,white,white,partial,white,one,pendant,chocolate,scattered,grasses,Bruises
4,1210,edible,convex,smooth,white,null,free,crowded,broad,black,...,white,white,partial,white,one,evanescent,black,abundant,grasses,No presenta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4598,1028,edible,convex,fibrous,brown,null,free,close,broad,purple,...,gray,gray,partial,white,one,pendant,black,several,wood,Bruises
4599,5591,poisonous,convex,smooth,red,fishy,free,close,narrow,buff,...,pink,white,partial,white,one,evanescent,white,several,wood,No presenta
4600,2053,edible,convex,fibrous,red,null,free,close,broad,brown,...,white,gray,partial,white,one,pendant,brown,solitary,wood,Bruises
4601,4604,poisonous,convex,scaly,brown,foul,free,close,narrow,buff,...,pink,pink,partial,white,one,evanescent,white,several,paths,No presenta


In [11]:
setas = pd.read_csv('data/train_setas.csv')

In [12]:
df = setas.copy()

In [13]:
df.set_index('Observacion',inplace=True)

In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        4603 non-null   object
 1   class                     4603 non-null   object
 2   cap_shape                 4603 non-null   object
 3   cap_surface               4603 non-null   object
 4   cap_color                 4603 non-null   object
 5   odor                      4603 non-null   object
 6   gill_attachment           4603 non-null   object
 7   gill_spacing              4603 non-null   object
 8   gill_size                 4603 non-null   object
 9   gill_color                4603 non-null   object
 10  stalk_shape               4603 non-null   object
 11  stalk_root                4603 non-null   object
 12  stalk_surface_above_ring  4603 non-null   object
 13  stalk_surface_below_ring  4603 non-null   object
 14  stalk_color_above_ring  

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4603 entries, 0 to 6575
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   clase                     4603 non-null   object
 1   cap-shape                 4603 non-null   object
 2   cap-surface               4603 non-null   object
 3   cap-color                 4603 non-null   object
 4   odor                      2133 non-null   object
 5   gill-attachment           4603 non-null   object
 6   gill-spacing              4603 non-null   object
 7   gill-size                 4603 non-null   object
 8   gill-color                4603 non-null   object
 9   stalk-shape               4603 non-null   object
 10  stalk-root                3336 non-null   object
 11  stalk-surface-above-ring  4603 non-null   object
 12  stalk-surface-below-ring  4603 non-null   object
 13  stalk-color-above-ring    4603 non-null   object
 14  stalk-color-below-ring    460

In [15]:
df.shape

(4603, 23)

In [21]:
df2.columns

Index(['id', 'class', 'cap_shape', 'cap_surface', 'cap_color', 'odor',
       'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
       'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
       'stalk_surface_below_ring', 'stalk_color_above_ring',
       'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
       'ring_type', 'spore_print_color', 'population', 'habitat', 'bruises'],
      dtype='object')

In [16]:
df.columns

Index(['clase', 'cap-shape', 'cap-surface', 'cap-color', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat', 'bruises'],
      dtype='object')

In [36]:
df2[df2['odor']=='null']

,id,class,cap_shape,cap_surface,cap_color,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat,bruises
2,2864,edible,convex,fibrous,green,null,free,close,broad,pink,...,pink,white,partial,white,one,pendant,brown,several,wood,Bruises
4,1210,edible,convex,smooth,white,null,free,crowded,broad,black,...,white,white,partial,white,one,evanescent,black,abundant,grasses,No presenta
8,14,edible,sunken,fibrous,green,null,free,close,narrow,black,...,white,white,partial,white,one,pendant,brown,solitary,urban,No presenta
9,2540,edible,convex,scaly,brown,null,free,close,broad,purple,...,gray,gray,partial,white,one,pendant,brown,solitary,wood,Bruises
10,2851,edible,convex,scaly,red,null,free,close,broad,pink,...,gray,gray,partial,white,one,pendant,brown,solitary,wood,Bruises
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,180,edible,flat,fibrous,green,null,free,close,narrow,pink,...,white,white,partial,white,one,pendant,black,several,urban,No presenta
4594,3770,edible,flat,fibrous,red,null,free,close,broad,purple,...,white,gray,partial,white,one,pendant,brown,several,wood,Bruises
4595,4111,edible,flat,scaly,buff,null,free,close,broad,white,...,white,white,partial,white,two,evanescent,white,clustered,waste,Bruises
4598,1028,edible,convex,fibrous,brown,null,free,close,broad,purple,...,gray,gray,partial,white,one,pendant,black,several,wood,Bruises


In [35]:
df.loc[2864]

clase                         edible
cap-shape                     convex
cap-surface                  fibrous
cap-color                      green
odor                             NaN
gill-attachment                 free
gill-spacing                   close
gill-size                      broad
gill-color                      pink
stalk-shape                 tapering
stalk-root                   bulbous
stalk-surface-above-ring      smooth
stalk-surface-below-ring      smooth
stalk-color-above-ring          pink
stalk-color-below-ring        white.
veil-type                    partial
veil-color                     white
ring-number                      one
ring-type                    pendant
spore-print-color              brown
population                   several
habitat                         wood
bruises                     Presenta
Name: 2864, dtype: object

In [22]:
df2.isnull().sum()

id                          0
class                       0
cap_shape                   0
cap_surface                 0
cap_color                   0
odor                        0
gill_attachment             0
gill_spacing                0
gill_size                   0
gill_color                  0
stalk_shape                 0
stalk_root                  0
stalk_surface_above_ring    0
stalk_surface_below_ring    0
stalk_color_above_ring      0
stalk_color_below_ring      0
veil_type                   0
veil_color                  0
ring_number                 0
ring_type                   0
spore_print_color           0
population                  0
habitat                     0
bruises                     0
dtype: int64

In [17]:
df.isnull().sum()

clase                          0
cap-shape                      0
cap-surface                    0
cap-color                      0
odor                        2470
gill-attachment                0
gill-spacing                   0
gill-size                      0
gill-color                     0
stalk-shape                    0
stalk-root                  1267
stalk-surface-above-ring       0
stalk-surface-below-ring       0
stalk-color-above-ring         0
stalk-color-below-ring         0
veil-type                      0
veil-color                     0
ring-number                   15
ring-type                     15
spore-print-color              0
population                     0
habitat                        0
bruises                        0
dtype: int64